In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing, impute
from IPython.display import display

from sklearn.dummy import DummyRegressor
import lightgbm as lgb
import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_val_predict, cross_validate
from sklearn.model_selection import train_test_split, GroupKFold
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.linear_model import Ridge, RidgeCV

from fbprophet import Prophet
import optuna
import statsmodels.api as sm
import scipy.stats as stats

plt.style.use('ggplot')


In [ ]:
random_state = 7014
rng = np.random.RandomState(random_state)

In [ ]:
X = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/train.csv', index_col=0)
X_test = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/test.csv', index_col=0)

In [ ]:
df = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/train.csv', index_col=0)

In [ ]:
print(X.columns.difference(X_test.columns))
y = X.pop('pressure')

In [ ]:
X['time_step']

In [ ]:
X.columns

In [ ]:
X.shape

In [ ]:
X.describe()

In [ ]:
X.isna().sum()

In [ ]:
plt.figure(figsize=(12,5))
plt.title("Distribution of pressure")
ax = sns.distplot(y)



In [ ]:
df.pressure

In [ ]:
#pip install scipy

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as st
import statsmodels as sm
import warnings
# Matplotlib e Seaborn
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="white", palette="muted", color_codes=True)
# Plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode()

In [ ]:
print("p-value for pressure distribution: {}".format(st.normaltest(df.pressure.values)[1]))
plt.figure(figsize=(12,5))
plt.title("Distribution of pressure vs best fit normal distribution")
ax = sns.distplot(df.pressure, fit= st.norm, kde=True, color='g')



In [ ]:
df

In [ ]:
# Code (function) adapted from https://stackoverflow.com/questions/6620471/fitting-empirical-distribution-to-theoretical-ones-with-scipy-python
def best_fit_distribution(data, bins= 200):
    """Model data by finding best fit distribution to data"""
    y, x = np.histogram(data, bins=bins, density=True)
    x = (x + np.roll(x, -1))[:-1] / 2.0

    DISTRIBUTIONS = [        
        st.alpha,st.beta,st.chi,st.chi2, st.dgamma,st.dweibull,st.erlang,st.exponweib,
        st.f, st.genexpon,st.gausshyper,st.gamma, st.johnsonsb,st.johnsonsu, st.norm,
        st.rayleigh,st.rice,st.recipinvgauss, st.t, st.weibull_min,st.weibull_max
    ]

    best_distribution = st.norm
    best_params = (0.0, 1.0)
    best_sse = np.inf

    for distribution in DISTRIBUTIONS:
        #print("Testing " + str(distribution))

        # Try to fit the distribution
        #try:
        # Ignore warnings from data that can't be fit
        with warnings.catch_warnings():
            warnings.filterwarnings('ignore')

            # fit dist to data
            params = distribution.fit(data)

            # Separate parts of parameters
            arg = params[:-2]
            loc = params[-2]
            scale = params[-1]

            # Calculate fitted PDF and error with fit in distribution
            pdf = distribution.pdf(x, loc=loc, scale=scale, *arg)
            sse = np.sum(np.power(y - pdf, 2.0))

            # identify if this distribution is better
            if best_sse > sse > 0:
                best_distribution = distribution
                best_params = params
                best_sse = sse
        #except Exception:
        #    pass

    return (best_distribution.name, best_params)

dist_name, best_params = best_fit_distribution(df.pressure.values)
print("Best distribution found: {}, with parameters: {}".format(dist_name, best_params))

In [ ]:
df.pressure

In [ ]:
plt.figure(figsize=(12,5))
plt.title("Distribution of pressure vs Johnson-SB distribution (best fit)")
ax = sns.distplot(df.pressure, fit= st.johnsonsb, kde=True, color='g')